In [1]:
import pandas as pd
import fastai
import matplotlib.pyplot as plt
import sklearn as sk
import os
import time 
from datetime import datetime

import plotly as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

# print(os.listdir('/Users/aniruddhaprabhu/Library/Mobile Documents/com~apple~CloudDocs/'))
tweets = pd.read_csv('/Users/aniruddhaprabhu/Library/Mobile Documents/com~apple~CloudDocs/Data/trump/csv/djt-11032019.csv')

In [2]:
tweets.head()

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter for iPhone,Don’t miss this show! https://t.co/EhY8CNAUqP,Mon Nov 04 01:08:27 +0000 2019,4943,15904,False,1191160243784433665
1,Twitter for iPhone,False stories are being reported that a few Re...,Mon Nov 04 00:59:45 +0000 2019,9725,37358,False,1191158055397969920
2,Twitter for iPhone,....should give their own transcripts of the i...,Mon Nov 04 00:48:55 +0000 2019,6717,23427,False,1191155326743195648
3,Twitter for iPhone,"If Shifty Adam Schiff, who is a corrupt politi...",Mon Nov 04 00:48:51 +0000 2019,8344,28369,False,1191155312084082688
4,Twitter for iPhone,https://t.co/3O0ABd3HGt,Sun Nov 03 22:24:47 +0000 2019,9543,26444,False,1191119056910372864


In [3]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28591 entries, 0 to 28590
Data columns (total 7 columns):
source            28591 non-null object
text              28591 non-null object
created_at        28591 non-null object
retweet_count     28591 non-null int64
favorite_count    28591 non-null int64
is_retweet        28533 non-null object
id_str            28591 non-null int64
dtypes: int64(3), object(4)
memory usage: 1.5+ MB


In [4]:
sources = tweets['source'].value_counts().to_frame()

fig = go.Figure(data=[go.Pie(labels=sources.index.values, values=sources['source'])])
fig.update_layout(
    title_text="Tweet Sources for Donald Trump")
fig.show()

In [5]:
tweets['created_at'] = pd.to_datetime(tweets['created_at'], format='%a %b %d %H:%M:%S %z %Y')
tweets['created_at'].head()

0   2019-11-04 01:08:27+00:00
1   2019-11-04 00:59:45+00:00
2   2019-11-04 00:48:55+00:00
3   2019-11-04 00:48:51+00:00
4   2019-11-03 22:24:47+00:00
Name: created_at, dtype: datetime64[ns, UTC]

The times indicated above are from UTC+0000/Greenwich Median Time/U.K Time. Since Donald Trump resides in the White House and conducts his business around D.C./New York/Florida, it is best to convert those times to Eastern Standard Time/UTC-0500 as a frame of reference for when he tweets.

In [6]:
tweets['created_at'] = tweets['created_at'].dt.tz_convert('US/Eastern')
tweets['created_at'].head()

0   2019-11-03 20:08:27-05:00
1   2019-11-03 19:59:45-05:00
2   2019-11-03 19:48:55-05:00
3   2019-11-03 19:48:51-05:00
4   2019-11-03 17:24:47-05:00
Name: created_at, dtype: datetime64[ns, US/Eastern]

In [7]:
days = tweets['created_at'].dt.day_name().value_counts().to_frame()

# fig = go.Figure(data=[go.Pie(labels=days.index.values, values=days['created_at'])])
# fig.update_layout(
#     title_text="Trump's Favorite Tweet Days")
# fig.show()

fig = go.Figure(data=[go.Bar(x=days.index.values, y=days['created_at'])])
fig.update_layout(
    title_text="Trump's Favorite Tweet Days")
fig.show()

It's pretty obvious to see here that Mr. Trump tweets pretty evenly on the weekdays, but tends to cut down on the weekends. 